In [20]:
# Reproduce book environment
import random
random.seed(1234)

import logging
from pprint import pprint
from sys import stdout as STDOUT

## 項目 70 最適化の前にプロファイル

Pythonの動的な性質は、実行時の性能に関して驚くべき振る舞いを引き起こすことがあります。遅いはずだと想定した演算(文字列操作、ジェネレータなど)が実際には非常に速いことがあります。速いと想定した言語機能(属性アクセス、関数呼び出しなど) が実際には非常に遅いことがあります。Python プログラムで遅くなった真の原因を把握することは難しいのです。

最良の方策は、直観を無視して、プログラムを最適化する前に、その性能を直接測ることです。Python は、組み込みのプロファイラを提供していて、プログラムのどの部分が性能に責任あるかを決定できます。これによって、最適化の努力を問題が最も大きい部分に集中し、速度に影響がない部分を無視することができます (すなわち、アムダールの法則に従う)。

例えば、なぜ、このプログラムのアルゴリズムが遅いのかを突き止めたいとします。挿入ソートを使ってデータのリストをソートする関数を定義します。

In [21]:
# Example 1
def insertion_sort(data):
    result = []
    for value in data:
        insert_value(result, value)
    return result

挿入ソートでコアとなる機構はデータの各々について挿入点を見つける関数です。入力配列を線形
走査する、非常に効率の悪い関数 insert_value を次のように定義します。

In [22]:
# Example 2
def insert_value(array, value):
    for i, existing in enumerate(array):
        if existing > value:
            array.insert(i, value)
            return
    array.append(value)

insertion_sort と insert_value のプロファイルを取るために、乱数のデータを作り、プロファイラに渡す test 関数を定義します。

In [23]:
# Example 3
from random import randint

max_size = 10**4
data = [randint(0, max_size) for _ in range(max_size)]
test = lambda: insertion_sort(data)

Python は、2つの組み込みプロファイラを提供します。1つは、ピュア Python (profile) で、もう 1つはC拡張モジュール(cProfile) です。組み込みモジュール cProfile のほうが、プロファイル を取っているときに、プログラムの性能への影響が最小なので、適しています。ピュアPythonのだと オーバーヘッドが大きくて、 結果に歪みが出ます。

★ Python プログラムのプロファイルを取るとき、測定しているのがコードそのもので、 外部 システムではないことを確認すること。 ネットワークやディスク上の資源にアクセスする 関数に気をつけること。 それらは、その基盤システムそれ自体の遅さによって、 プログラ by ムの実行時間に大きな影響を及ぼす。 プログラムがそれらの遅い資源の遅延をカバーする ためにキャッシュを使っているなら、 プロファイルを取る前に、 十分にウォームアップして キャッシュの状態が大丈夫なことを確かめること。

cProfile モジュールから profile オブジェクトをインスタンス化して、runcall メソッドを使ってテスト関数を実行します。

In [24]:
# Example 4
from cProfile import Profile

profiler = Profile()
profiler.runcall(test)


[0,
 2,
 3,
 3,
 5,
 8,
 9,
 10,
 11,
 12,
 12,
 12,
 14,
 16,
 17,
 17,
 17,
 19,
 19,
 23,
 23,
 24,
 25,
 26,
 27,
 27,
 29,
 29,
 30,
 31,
 31,
 32,
 33,
 40,
 41,
 41,
 44,
 45,
 45,
 47,
 47,
 48,
 51,
 52,
 52,
 55,
 56,
 62,
 62,
 63,
 64,
 65,
 66,
 66,
 66,
 66,
 67,
 68,
 69,
 71,
 71,
 74,
 74,
 75,
 75,
 77,
 78,
 78,
 81,
 82,
 84,
 84,
 88,
 88,
 88,
 89,
 90,
 90,
 90,
 91,
 92,
 93,
 94,
 96,
 99,
 101,
 102,
 104,
 105,
 105,
 108,
 109,
 110,
 111,
 112,
 112,
 113,
 114,
 114,
 120,
 121,
 122,
 122,
 123,
 124,
 125,
 126,
 126,
 126,
 129,
 129,
 132,
 133,
 133,
 134,
 134,
 136,
 138,
 139,
 140,
 140,
 141,
 142,
 143,
 143,
 143,
 143,
 145,
 145,
 150,
 151,
 151,
 151,
 154,
 154,
 155,
 156,
 158,
 159,
 160,
 161,
 161,
 162,
 165,
 166,
 167,
 168,
 168,
 170,
 170,
 171,
 171,
 172,
 172,
 172,
 174,
 176,
 177,
 177,
 181,
 181,
 181,
 184,
 184,
 185,
 186,
 187,
 188,
 190,
 191,
 192,
 193,
 193,
 193,
 194,
 195,
 195,
 200,
 201,
 205,
 205,
 205,


テスト関数の実行が終わったら、組み込みモジュール pstatsとそのStatsクラスを使って性能に ついての統計情報を取り出せます。 Stats オブジェクトのさまざまなメソッドがプロファイル情報を どのように選択ソートして、必要なものだけを表示するように調整するのをサポートします。

In [25]:
# Example 5
from pstats import Stats

stats = Stats(profiler)
stats = Stats(profiler, stream=STDOUT)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()


         20003 function calls in 0.950 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.950    0.950 757775735.py:6(<lambda>)
        1    0.003    0.003    0.950    0.950 587521532.py:2(insertion_sort)
    10000    0.941    0.000    0.948    0.000 3845090190.py:2(insert_value)
     9992    0.006    0.000    0.006    0.000 {method 'insert' of 'list' objects}
        8    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




この出力は、 関数ごとの情報の表です。 データサンプルは、プロファイラが先ほどのruncallメソッ
ドの中でアクティブになっている間だけ収集されたものです。


プロファイラの統計カラムの意味について簡単に解説します。
- ncalls
  - プロファイル期間に関数が呼ばれた回数。
- tottime
  - 他の関数呼び出しに費やした時間を除いた関数実行に費やした秒数。
- tottime percall
  - 他の関数呼び出しに費やした時間を除いた、関数が1回当たり呼び出されて実行に要した平均秒数。tottime をncallsで割ったもの。
- cumtime
  - それが呼び出している関数の実行に費やした時間も含めた関数実行に要した累積秒数。
- cumtime percall
  - 呼び出している関数の実行に費やした時間も含めた関数が1回当たりに呼び出されて実行に要した平均秒数。

上のプロファイラの統計表を見れば、テストでCPUの最大消費が insert_value 関数で費やした累積時間だとわかります。この関数をモジュール bisect を使って再定義します(「項目72 ソート済
みシーケンスの探索には bisect を考える」参照)。


In [26]:
# Example 6
from bisect import bisect_left

def insert_value(array, value):
    i = bisect_left(array, value)
    array.insert(i, value)

プロファイラを再度実行して新しいプロファイラ統計の表を生成します。新しい関数はずっと速く累積時間は修正前の insert_value 関数よりもほぼ100倍小さくなっています。


場合によると、全体のプログラムのプロファイルを取っているときに、実行時間の大半の責任を共通のユーティリティ関数が負っていることがわかります。 プロファイラのデフォルトの出力では、ユーティリティ関数がプログラムの多くの異なった部分からどのように呼ばれているかを示さないので、この状況の理解が難しいのです。

例えば、my_utility 関数がプログラム中の2つの異なる関数から繰り返し呼ばれている場合です。

In [27]:
# Example 8
def my_utility(a, b):
    c = 1
    for i in range(100):
        c += a * b

def first_func():
    for _ in range(1000):
        my_utility(4, 5)

def second_func():
    for _ in range(10):
        my_utility(1, 3)

def my_program():
    for _ in range(20):
        first_func()
        second_func()

このコードのプロファイルを取り、デフォルトのprint_stats出力を使って出力した統計はよくわかりません。

In [28]:
# Example 9
profiler = Profile()
profiler.runcall(my_program)
stats = Stats(profiler, stream=STDOUT)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()

         20242 function calls in 0.073 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.073    0.073 3510307165.py:15(my_program)
       20    0.003    0.000    0.072    0.004 3510307165.py:7(first_func)
    20200    0.070    0.000    0.070    0.000 3510307165.py:2(my_utility)
       20    0.000    0.000    0.001    0.000 3510307165.py:11(second_func)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




my_utility 関数が明らかに実行時間の大半を占めますが、なぜこの関数がそれほど呼ばれているのかがすぐには明らかになりません。プログラムのコードを探せば、my_utility に複数の呼び出し元のあることがわかりますが、それでもまだはっきりしないでしょう。

こういうことを処理するために、Pythonプロファイラは、各関数のプロファイル情報に、どの呼び
出し元が貢献しているかを表示する方法を提供しています。

In [29]:
# Example 10
stats = Stats(profiler, stream=STDOUT)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_callers()

   Ordered by: cumulative time

Function                                          was called by...
                                                      ncalls  tottime  cumtime
3510307165.py:15(my_program)                      <- 
3510307165.py:7(first_func)                       <-      20    0.003    0.072  3510307165.py:15(my_program)
3510307165.py:2(my_utility)                       <-   20000    0.070    0.070  3510307165.py:7(first_func)
                                                         200    0.001    0.001  3510307165.py:11(second_func)
3510307165.py:11(second_func)                     <-      20    0.000    0.001  3510307165.py:15(my_program)
{method 'disable' of '_lsprof.Profiler' objects}  <- 




今度のプロファイラ統計表では、呼ばれた関数が左に、 呼び出しの責任を持つ側が右に示されます。 これで、my_utility が first_func でほとんど使われていることが明らかになります。

### 覚えておくこと

- 速度低下の原因がよくわかっていないことが多いので、最適化の前にPython プログラムの
プロファイルを取ることが重要だ
- より正確なプロファイル情報が得られるので、profile モジュールではなく cProfile モ
ジュールを使う
- Profile オブジェクトのruncallメソッドは、関数呼び出しの木を隔離してプロファイル
を取るために必要なすべてを提供する
- Stats オブジェクトは、プログラムの性能を理解するために知る必要のあるプロファイル情報の部分集合を選び出して出力する